In [0]:
#importing
import numpy
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
import random

In [0]:
final_img = plt.imread('/content/fighter.jpeg')
if len(final_img.shape) == 2:  # image is grayscale
    final_img = numpy.stack((final_img,)*3, axis=-1)  # convert to RGB
initial_img = numpy.zeros((512,512,3))

In [0]:
final_sum = numpy.sum(final_img)
#fitness function
def fitness(initial_img_array):
    #my fitness function is the sum of absolute difference between the two images
    return 1 / numpy.sum(numpy.abs(final_img - initial_img_array))

In [0]:
#rectangles
common = [(109,123,166),
          (166,135, 78),
          (217,191,143),
          (115, 89, 57),
          (155,164,191),
          (217,193,180),
          (199,123,166)]
def draw_rectangle(initial_img, width=10, height=10):
    #upperleft
    x0 = random.randint(0, 512 - width)
    y0 = random.randint(0, 512 - height)
    #upperright
    x1 = x0 + width
    y1 = y0
    #lowerleft
    x2 = x0
    y2 = y0 + height
    #lowerright
    x3 = x1
    y3 = y1 + height

    #colour
#     if random.randint(0,100) < 30:
#         c = random.choice(common)
#     else:
    c0 = random.randint(0, 512)
    c1 = random.randint(0, 512)
    c2 = random.randint(0, 512)
    c = (c0, c1, c2)

    copy = Image.fromarray(numpy.uint8(initial_img*255), 'RGB')
    ImageDraw.Draw(copy).rectangle([(x0,y0), (x3,y3)], fill=c)
    return numpy.asarray(copy)

In [0]:
#initializing population
def populate(size=10, rectangles=2, width=30, height=30):
    population = []
    for _ in range(size):
        specie = initial_img
        for _ in range(rectangles):
            specie = draw_rectangle(specie, width=width, height=height)
        population.append(specie)
    return population

In [0]:
#crossover Uniform function
def crossover(p1, p2, uniform=False):
    assert p1.shape == p2.shape
    if uniform:
        take = numpy.random.randint(low=0, high=2, size=p1.shape[:2])[..., numpy.newaxis]
        child1 = take * p1 + (1-take) * p2
        child2 = (1-take) * p1 + take * p2
    else:
        prob = random.random()
        size = min(len(p1), len(p2))
        child1 = p1.copy()
        child2 = p2.copy()
        for i in range(size):
            if random.random() < prob:
                child1[i], child2[i] = child2[i], child1[i]
    return child1, child2

In [0]:
def mutate(img, rectangles=3, width=40, height=40):
    for _ in range(rectangles):
        img = draw_rectangle(img, width=width, height=height)
    return img

In [0]:
population = populate(size=30, rectangles=3)
gens = 5000000

for i in range(gens):
    #calculating fitness function for each
    fits = []
    for specie in population:
        score = fitness(specie)
        fits.append((specie, score))
    #sorting
    sorted_pop = sorted(fits, key=lambda x: x[1])

    #selecting top 5
    best_pop = sorted_pop[:5] # population size = 2*k^2
    images, scores = zip(*best_pop)
  
    if i%100 == 0:
      print('Generation', i)
      plt.imshow(images[0])
      plt.show()
      print(scores[0])

    children_array = []
    for img1 in images:
        for img2 in images:
            child1, child2 = crossover(img1, img2, uniform=True)
            children_array.append(child1)
            children_array.append(child2)

    #mutating
    mutated = []
    for child in children_array:
        mutated_one = mutate(child, rectangles=1, width=50//(i//200+1), height=50//(i//200+1))
        mutated.append(mutated_one)
    population = mutated